In [1]:
import os

import pandas as pd
from Bio import SeqIO

In [2]:
dataDir = "Data"

In [3]:
df = pd.read_csv(os.path.join(dataDir, "meta_data_0212.csv"))
df["Collection date"] = pd.to_datetime(df["Collection date"])

In [4]:
genomeTreeDir = "Genome_tree"
if not os.path.exists(genomeTreeDir):
    os.mkdir(genomeTreeDir)

# excluded as agreed http://virological.org/t/phylodynamic-analysis-90-genomes-12-feb-2020/356/4
excluded = ["EPI_ISL_406592", "EPI_ISL_406595"]

outgroup = None
earlies = None
outputSeqs = []
outputSeqs2 = []

for record in SeqIO.parse(os.path.join(dataDir, "Seq_combine_0211.fasta"), "fasta"):
    if record.id != "ZYZ":
        ac = record.id.split('|')[-1]
        cd = df[df["Accession ID"] == ac]["Collection date"].values[0]
        if outgroup is None:
            outgroup = ac
            earlies = cd
        elif cd < earlies:
            outgroup = ac
            earlies = cd
        record.id = ac
        record.description = ""
        outputSeqs.append(record)
        if ac not in excluded:
            outputSeqs2.append(record)

with open(os.path.join(genomeTreeDir, "outgroup.txt"), 'w') as f:
    f.write(outgroup + "\n")
SeqIO.write(outputSeqs, os.path.join(genomeTreeDir, "raw.fasta"), "fasta")
SeqIO.write(outputSeqs2, os.path.join(genomeTreeDir, "raw2.fasta"), "fasta")

89